In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn import tree

mallards=pd.read_excel("Mallards_initial data.xlsx")

In [2]:
print(mallards.head(5))

   population  farmed       date       hour  slatitude  slongitude
0          73       0 2012-09-12   15:32:58  56.440033   13.992838
1          73       0 2012-09-12   16:32:22  56.439972   13.992728
2          73       0 2012-09-12   17:29:51  56.440144   13.992794
3          73       0 2012-09-12   18:28:04  56.440147   13.992643
4          73       0 2012-09-14   15:26:23  56.439632   13.992536


In [3]:
ordered = mallards.sort_values(by=["population"])

In [4]:
ordered.head(10)

,population,farmed,date,hour,slatitude,slongitude
4330,1,0,2012-09-04,10:22:35,56.439278,13.991564
4278,1,0,2012-08-31,17:57:56,56.439716,13.991791
4279,1,0,2012-08-31,18:56:34,56.440170,13.992670
4280,1,0,2012-08-31,19:55:23,56.440136,13.992191
4281,1,0,2012-08-31,20:54:15,56.441143,13.988527
4282,1,0,2012-08-31,21:54:04,56.439919,13.991972
4283,1,0,2012-08-31,22:53:11,56.439869,13.992161
4284,1,0,2012-08-31,23:50:06,56.439716,13.992242
4285,1,0,2012-09-01,00:49:04,56.439934,13.991963
4286,1,0,2012-09-01,01:46:39,56.439888,13.992043


In [6]:
ordered.to_excel("mallards_ordered.xlsx", index=False)

In [10]:
labelled = pd.DataFrame(columns=["dlatitude", "dlongitude"])
for i, row in ordered.iterrows():
    if i == ordered.index[-1]:
        break
    if row["population"] != ordered.iloc[i+1]["population"]:
        label_row = pd.DataFrame({"dlatitude": row["slatitude"], "dlongitude": row["slongitude"]}, index=[i])
    else:
        label_row = pd.DataFrame({"dlatitude": ordered.iloc[i+1]["slatitude"], "dlongitude": ordered.iloc[i+1]["slongitude"]}, index=[i])
    labelled = pd.concat([labelled, label_row])

In [11]:
labelled.to_excel("mallards_labels.xlsx", index=False)

In [12]:
labelled.head(10)

,dlatitude,dlongitude
0,56.439716,13.991791
1,56.440170,13.992670
2,56.440136,13.992191
3,56.441143,13.988527
4,56.439919,13.991972
5,56.439869,13.992161
6,56.439716,13.992242
7,56.439934,13.991963
8,56.439888,13.992043
9,56.440109,13.991787
